In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
svp_clusters = dataiku.Folder("hAbB1mJR")
svp_clusters_info = svp_clusters.get_info()


# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

#hyperparameter_Tuning_df = ... # Compute a Pandas dataframe to write into Hyperparameter_Tuning


# # Write recipe outputs
# hyperparameter_Tuning = dataiku.Dataset("Hyperparameter_Tuning")
# hyperparameter_Tuning.write_with_schema(hyperparameter_Tuning_df)

In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow as tf
import cv2
import os
import numpy as np
from PIL import Image
import matplotlib as plt
# Read recipe inputs
svp_clusters = dataiku.Folder("hAbB1mJR")
svp_clusters_info = svp_clusters.get_info()


# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

# cnn_result_df = ... # Compute a Pandas dataframe to write into CNN_result


# # Write recipe outputs
# cnn_result = dataiku.Dataset("CNN_result")
# cnn_result.write_with_schema(cnn_result_df)

In [0]:
import os
conn_folder = dataiku.Folder('hAbB1mJR')
all_files_paths = conn_folder.list_paths_in_partition()
image_paths = [file_path for file_path in all_files_paths if file_path.endswith(".jpg")]
label_names = []
resized_images = []
for idx ,image_path in enumerate(image_paths):
    label_name = os.path.basename(image_path)[:2]
    pil_image=Image.open(conn_folder.get_download_stream(image_path))
    #resized_img = pil_image.resize((20,20))
    resized_images.append(pil_image)
    label_names.append(label_name)
    if idx == 10:
        break

In [0]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
columns = 5
for i, image in enumerate(resized_images):
    plt.subplot(len(resized_images) / columns + 1, columns, i + 1)
    plt.imshow(image, cmap = 'gray')

In [0]:
import ipyplot

ipyplot.plot_images(resized_images, max_images=20, img_width=150)

In [0]:
from numpy import asarray
labels = ['protein', 'silicon']
img_size = 20

def get_data(image_paths):
    data = []
    labels = []
    sil_cnt = 0
    prot_cnt = 0
    for image_path in image_paths:
            try:
                label_name = os.path.basename(image_path)[:2]
                if label_name == 'p5':
                    class_num = 0
                    prot_cnt+=1
                    if prot_cnt > 1000:
                        continue
                else:
                    class_num = 1
                    sil_cnt+=1
                    if sil_cnt >1000:
                        continue
                if prot_cnt > 1000 and sil_cnt >1000:
                    break
                pil_image=Image.open(conn_folder.get_download_stream(image_path))
                resized_img = pil_image.resize((20,20))
                resized_img = np.expand_dims(resized_img, axis=-1)
                resized_arr = asarray(resized_img)
                data.append(resized_arr)
                labels.append(class_num)
            except Exception as e:
                print(e)

    return np.array(data),labels
X,y = get_data(image_paths)

In [0]:
import seaborn as sns
l = []
for i in y:
    if(i == 0):
        l.append("Protein")
    else:
        l.append("Silicon")
sns.set_style('darkgrid')
sns.countplot(l)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(X_train.shape)

In [0]:
import time
start=time.time()

# define a function to create model, required for KerasClassifier
# the function takes drop_out rate as argument so we can optimize it
def create_cnn_model(dropout_rate=0):
    model = Sequential()
    model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(20,20,1)))
    model.add(MaxPool2D())
    model.add(Conv2D(32, 3, padding="same", activation='relu'))
    model.add(MaxPool2D())
    model.add(Conv2D(64, 3, padding="same", activation='relu'))
    model.add(MaxPool2D())
    model.add(Dropout(rate = dropout_rate))
    model.add(Flatten())
    model.add(Dense(128,activation="sigmoid"))
    model.add(Dense(2, activation="softmax"))

    # Compile model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )
    return model

In [0]:
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))

In [0]:
# create model
from keras.utils import to_categorical
model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
# define parameters and values for grid search
param_grid = {
    'batch_size': [64, 128],
    'epochs': [50, 100,200],
    'dropout_rate': [0.10, 0.20],
    #'conv_activation' : ['sigmoid', 'tanh', 'relu']
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, to_categorical(y))  # fit the full dataset as we are using cross validation

# print out results
print('time for grid search = {:.0f} sec'.format(time.time()-start))
display_cv_results(grid_result)

In [0]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(20,20,1)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

opt = Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [0]:
history = model.fit(X_train,y_train, batch_size = 128, epochs = 200 , validation_data = (X_test, y_test))

In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(200)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Test Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Test Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Test Loss')
plt.legend(loc='upper right')
plt.title('Training and Test Loss')
plt.show()

In [0]:
predictions = model.predict_classes(X_test)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y_test, predictions, target_names = ['Protein','Silicon']))